<a href="https://colab.research.google.com/github/Dejan-Cugalj/MASTER-TEZA/blob/main/MASTER_TEZA_PLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **"Predikcija ispravnog očitavanje senzora mase protoka vazduha motora automobila"**
## *MASTER TEZA*
---

#### Mentori:
#### ***Prof. dr. Marina Marjanović Jakovljević***
####***Prof. dr. Miodrag Živković***
### Student: **Dejan Čugalj (2019/430054)**


---
#Prosta Linearna Regresija (PLR)
---

>### 1.0 - Učitavanje potrebnih biblioteka


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Sistemske
import datetime, os, time
# Numerika
import numpy as np
import pandas as pd
import pandas.util.testing as tm
from sklearn.preprocessing import StandardScaler
# Linearna Regresija
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
#Evaluacija
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import statsmodels.api as sm
#Grafika
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
print("*********** Učitane Biblioteke... ********************\n")

*********** Učitane Biblioteke... ********************



>### 2.0 - Učitavanje skupa podataka (eng *dataset*)

[Preuzimanje skupa podataka](https://github.com/Dejan-Cugalj/CANBUS_dataset/blob/main/golf-master-canbus_dataset.csv)


In [4]:
url = '/content/drive/MyDrive/FAKULTET/MASTER STUDIJE/MASTER TEZA-RAD/MATERIJAL/AI/1-PLR/golf-master-canbus_dataset.csv' # Uneti putanju do preuzetog dataseta
print("*********** Ucitan DATASET... ********************\n")

*********** Ucitan DATASET... ********************



>### 3. Preprocesuiranje podataka

> **LEGENDA DATASETA:**

*   **RPM** - **R**EVOLUTIONS **P**ER **M**INUTE sensor
*   **MAP** - **M**ANIFOLD **A**BSOLUTE **P**RESSURE sensor
*   **FGP** - **F**UEL **G**AUGE **P**RESSURE sensor
*   **IAT** - **I**NTAKE **A**IR **T**EMPERATURE sensor
*   **ECT** - **E**NGINE **C**OOLANT **T**EMPERATURE sensor
*   **CEL** - **C**ALCULATED **E**NGINE **L**OAD
*   **ABP** - **A**BSOLUTE **B**AROMETRIC **P**RESSURE
*   **MAF** - **M**ASS **A**IR **F**LOW sensor 








In [6]:

dataset = pd.read_csv(url ,sep=',', usecols = ['RPM', 'MAP', 'FGP', 'MAF'])
dataset.columns=['RPM', 'MAP', 'FGP', 'MAF']
 
print("*********** DATASET ********************\n")
print(dataset.head(10))

*********** DATASET ********************

      RPM  MAP    FGP    MAF
0  946.00   87  23580  11.42
1  947.00   87  23730  11.52
2  950.75   87  25820  11.87
3  945.50   87  24510  11.26
4  951.25   87  25050  11.38
5  950.00   87  26700  11.90
6  949.50   87  25870  10.98
7  949.00   87  26180  11.44
8  951.00   87  26810  11.05
9  949.25   87  25930  11.41


> #### 3.1 Kreiranje Pandas Data-Frejma

In [7]:
# Odbacivanje obeležija koje nisu u korelaciji
#dataset.drop(['IAT', 'ECT', 'CEL', 'ABP'], axis=1, inplace=True)
 
X = dataset.iloc[:, :-1].values # Alokacija Matrice Obeležija / Nezavisnih Promenjivih (eng. Independent variables)
y = dataset.iloc[:, -1].values # Alokacija Zavisne Promenjive (eng. Dependent variable) MAF sernzora automobila
print("**************************************************************")
print("**************** KREIRAN PANDAs DATA FREJM ******************")
print("**************************************************************\n")
print("*************************   X   ************************\n")
print(X)
print("\n\n\n*************************   y   ************************\n")
print(y)

**************************************************************
**************** KREIRAN PANDAs DATA FREJM ******************
**************************************************************

*************************   X   ************************

[[  946.      87.   23580.  ]
 [  947.      87.   23730.  ]
 [  950.75    87.   25820.  ]
 ...
 [  878.75    87.   24510.  ]
 [  879.25    87.   26730.  ]
 [  878.75    87.   23680.  ]]



*************************   y   ************************

[11.42 11.52 11.87 ...  9.41  9.5   9.57]
